In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

In [4]:
if True:
    with open("splitted_round_5.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_round_5_adjusted_only.pic", "rb") as r:
        traces_round_5 = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_round_5, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_round_5.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_round_5", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_round_5_adjusted.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_round_5_adjusted.keras")
except ValueError:
    pass

In [6]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_round_5_adjusted.keras", patience=10)

Epoch 1/1000
10547/10547 [==============================] - 1908s 180ms/step - loss: 41.2732 - block_perm_5_output_loss: 1.9913 - mask_5_0_0_output_loss: 2.8063 - mask_5_0_1_output_loss: 2.8070 - mask_5_1_0_output_loss: 2.8033 - mask_5_1_1_output_loss: 2.8041 - mask_5_2_0_output_loss: 2.8057 - mask_5_2_1_output_loss: 2.8068 - mask_5_3_0_output_loss: 2.8061 - mask_5_3_1_output_loss: 2.8038 - mask_5_4_0_output_loss: 2.8074 - mask_5_4_1_output_loss: 2.8061 - mask_5_5_0_output_loss: 2.8072 - mask_5_5_1_output_loss: 2.8053 - mask_5_6_0_output_loss: 2.8076 - mask_5_6_1_output_loss: 2.8049 - block_perm_5_output_accuracy: 0.1440 - mask_5_0_0_output_accuracy: 0.0626 - mask_5_0_1_output_accuracy: 0.0627 - mask_5_1_0_output_accuracy: 0.0626 - mask_5_1_1_output_accuracy: 0.0626 - mask_5_2_0_output_accuracy: 0.0623 - mask_5_2_1_output_accuracy: 0.0627 - mask_5_3_0_output_accuracy: 0.0620 - mask_5_3_1_output_accuracy: 0.0619 - mask_5_4_0_output_accuracy: 0.0622 - mask_5_4_1_output_accuracy: 0.0624 -